In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import Auriga_Analysis as A
import Auriga_Analysis.Others_lowres as B
import imageio

plt.rcParams['text.usetex'] = True

In [1]:
%system free -m

['               total        used        free      shared  buff/cache   available',
 'Mem:         1546271      347877     1179364         547       27215     1198393',
 'Swap:           7475          45        7430']

In [ ]:
ids = B.load_snapshot(5, 100, 4, ["ParticleIDs"]).get_results()
position = B.group_data(5, 100, 0, 'Subhalo/SubhaloPos').return_detail()

In [ ]:
n = 18 #halo number
snap1 = 0
snap2 = 127
detail ='NextProgenitor'
tree1  = B.mergertree(n, snap2,snap1,0,0, detail)
print(B.time(n, snap2), B.time(n,snap1))

results, snaps4 = tree1.walk_the_tree(mainprogonly = True)

In [ ]:
masses4 = [mass[0][4] for mass in results]

In [ ]:
plt.plot(snaps1,masses1, label = '9')
plt.plot(snaps2, masses2, label = '5')
plt.plot(snaps3, masses3, label = '10')
plt.plot(snaps4, masses4, label = '18')

plt.legend()
plt.xlim(55,60)
plt.ylim(0.35,0.45)

In [ ]:
max(masses1)/10

In [ ]:
B.time(9,58)

In [ ]:
n = 17 #halo number
detail = 'SubhaloPos'
tree1  = B.mergertree(n, 127,0,0,0, detail)

results, snaps = tree1.walk_the_tree(mainprogonly = True)

In [ ]:
centres = np.array([result[0] for result in results])
snapshots = [snap[0] for snap in snaps]
times = [B.time(n, snap) for snap in snapshots]

#make ascending
times = times[::-1]
centres = centres[::-1]
snapshots = snapshots[::-1]

In [ ]:
plt.plot(times, centres[:,0])
plt.plot(times, centres[:,1])
plt.plot(times, centres[:,2])

In [ ]:
a = [B.scale_fac(n, s) for s in snapshots]

unsmoothed_velocity = A.Motion.use_splines(times, centres,a, 'peculiar_velocity')
smoothed = A.Motion.b_splines(4, 4, centres, times,a, 'peculiar_velocity')
smoothed_times = A.Motion.b_splines(4, 4, centres, times,a, 'times')

plt.figure(figsize = (5,4))

plt.plot(times, unsmoothed_velocity[0], label = 'x')
plt.plot(times, unsmoothed_velocity[1], label = 'y')
plt.plot(times, unsmoothed_velocity[2], label = 'z')

plt.plot(smoothed_times, smoothed[0], label = 'x smooth')
plt.plot(smoothed_times, smoothed[1], label = 'y smooth')
plt.plot(smoothed_times, smoothed[2], label = 'z smooth')

plt.xlim(0,14)
plt.ylabel('Velocity / km/s', fontsize = 12)
plt.xlabel('Time / Gyr', fontsize = 12)

plt.legend()
plt.title('Auriga {}'.format(n), fontsize = 14)

### Stellar density histogram

In [ ]:
n = 18 #halo number
Centres, snaps  = B.mergertree(n, 127,0,0,0, 'SubhaloPos').walk_the_tree(mainprogonly = True)
Radii, snaps  = B.mergertree(n, 127,0,0,0, 'Group_R_Crit200').walk_the_tree(mainprogonly = True)

centres = np.array([centre[0] for centre in Centres])
snapshots = [snap[0] for snap in snaps]
radii = np.array([radius[0] for radius in Radii])

In [ ]:
def stellar_hist(simulation_number, snap, centre, radial_cut, histogram_width):

    time = B.time(simulation_number, snap)
    
    b = histogram_width

    results =  B.load_snapshot(simulation_number, snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses"])
    coordinates, magnitudes, velocities, masses = results.get_results()
    
    # new_coords, new_vels, coords, vels, masses, magnitude = A.util.align(centre, coordinates, velocities, masses, radius, 0.5, idx = None, other = [magnitudes], to_print = False)

    if radius > 0:
        
        coordinates = A.util.CentreOnHalo(coordinates,centre)
        velocities, bulk_velocity = A.util.remove_bulk_velocity(coordinates, velocities, masses, idx = None, radialcut = radial_cut)
        star_radius = A.util.r(coordinates)
        
        istars, = np.where( (star_radius < radial_cut) )
        
        if len(istars,) == 0:
            print ('No stars in radialcut')
    
        else:
            L = np.cross( coordinates[istars,:], (velocities[istars,:] * masses[istars,None]) )
            Ltot = L.sum( axis=0 )
            Ldir = Ltot / np.sqrt( (Ltot**2).sum() )
            
            xdir, ydir, zdir = A.util.get_principal_axis( coordinates, masses, istars, L=Ldir )
            matrix = np.array( [xdir,ydir,zdir] )
            rotmat = matrix.transpose()
        
            # #for just stars in radial cut
            # new_coords = np.dot(coordinates[istars,:], rotmat)
            # new_vels = np.dot(velocities[istars,:], rotmat)
            # new_mass = masses[istars,]
        
            new_coords = np.dot(coordinates, rotmat)
            new_vels = np.dot(velocities, rotmat)
            new_mass = masses
            
            n, xedges, yedges = np.histogram2d(new_coords[:,1], new_coords[:,2], bins=(500,500), range = [[-b,b], [-b, b]])
            xbin = 0.5 * (xedges[:-1] + xedges[1:])
            ybin = 0.5 * (yedges[:-1] + yedges[1:])
            xc, yc = np.meshgrid(xbin, ybin)
            fig, ax1 = plt.subplots(figsize=(4,4))
            ax1.set_aspect(1)
            ax1.contourf( xc*1000, yc*1000, np.log10(n.T), cmap='magma')
        
            plt.suptitle('Auriga {}'.format(simulation_number), fontsize = 14)
            plt.title('Time = {} Gyr'.format(np.round(time,2)), fontsize = 12)
            plt.xlabel('kpc', fontsize = 12)
            plt.ylabel('kpc', fontsize = 12)


In [ ]:
filenames = []
count = 0

for i in range(20,127,3):
    snap = i

    index = snapshots.index(snap)
    centre = centres[index]
    radius = radii[index] 
    
    stellar_hist(n, snap,centre, 0.01, 0.035)
    
    name = 'Images_for_gif/{}.png'.format(count)
    plt.savefig(name)
    filenames.append(name)
    count+=1
    plt.close()

In [ ]:
with imageio.get_writer('radialcut_10.gif', mode='I', fps = 2) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

### Annulus to find center of light

In [ ]:
def load_aligned_data(n, snap):
    index = snapshots.index(snap)
    centre = centres[index]
    radius = radii[index]
    
    results =  B.load_snapshot(n, snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses"])
    coordinates, magnitudes, velocities, masses = results.get_results()
    coordinates = A.util.CentreOnHalo(coordinates,centre)

    if radius <=0:
        print('Undefined radius, using last known radius')
        radius = radii[-1]
        
    velocities, bulk_velocity = A.util.remove_bulk_velocity(coordinates, velocities, masses, idx = None, radialcut = 0.1*radius)
    star_radius = A.util.r(coordinates)
    
    istars, = np.where((star_radius < 0.1*radius))
    
    if len(istars,) == 0:
        print ('No stars in radialcut')
    else:
        L = np.cross( coordinates[istars,:], (velocities[istars,:] * masses[istars,None]) )
        Ltot = L.sum( axis=0 )
        Ldir = Ltot / np.sqrt( (Ltot**2).sum() )
        
        xdir, ydir, zdir = A.util.get_principal_axis( coordinates, masses, istars, L=Ldir )
        matrix = np.array( [xdir,ydir,zdir] )
        rotmat = matrix.transpose()

        new_coords = np.dot(coordinates, rotmat)
        new_vels = np.dot(velocities, rotmat)
        coord2d = new_coords[:,1:3]

    return coord2d, new_vels, magnitudes, masses

In [ ]:
def load_simulation_data(n):
    Centres, snaps  = B.mergertree(n, 127,0,0,0, 'SubhaloPos').walk_the_tree(mainprogonly = True)
    Radii, snaps  = B.mergertree(n, 127,0,0,0, 'Group_R_Crit200').walk_the_tree(mainprogonly = True)
    stellar_masses, snaps = B.mergertree(n, 127,0,0,0, 'SubhaloMassType').walk_the_tree(mainprogonly = True)
    half_mass, snaps = B.mergertree(n, 127,0,0,0, 'SubhaloHalfmassRadType').walk_the_tree(mainprogonly = True)
    
    centres = np.array([centre[0] for centre in Centres])
    snapshots = [snap[0] for snap in snaps]
    radii = np.array([radius[0] for radius in Radii])
    stellar_mass = np.array([mass[0][4] for mass in stellar_masses])   
    stellar_radius = np.array([rad[0][4] for rad in half_mass])


    return centres, snapshots, radii, stellar_mass, stellar_radius

In [ ]:
def snapshot_10_percent(masses, snapshots):
    tol = 0
    values = [[]]
    while len(values[0]) <1:
        tol+= 0.05
        values = np.where(np.isclose(masses, masses[0]/10, tol)== True)
    snap = snapshots[values[0][0]]
    return snap

In [ ]:
### WHAT DO STARTING VALUES OF ASSEMBLY LOOK LIKE?

simulations = [2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
stellar_mass = np.load('Data/stellarmass_divH.npy')
starting_snaps = np.load('Data/Starting_values.npy')

#for simulation 28, subtract by 10
starting_snaps[-1]-=10

new_sim = [2,3,5,6,7,8,9,10,12,14,15,16,17,18,19,21,22,23,24,25,26,27,28]

times = [B.time(1, snap) for snap in starting_snaps]

plt.figure(figsize = (5,5))
           # , dpi = 300)
plt.scatter(stellar_mass, times, marker = "o", s = 12, color = 'white')

for i, (x, y) in enumerate(zip(stellar_mass, times)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor='none', edgecolor='black', boxstyle='circle,pad=0.3'))

plt.xlabel(r'$\mathrm{M_* \,/\, 10^{10} \, M_\odot}$', fontsize = 15)
plt.ylabel(r'$\mathrm{Assembly\, \, time \, / \, Gyr}$', fontsize = 15)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

In [ ]:
### Find maximum displacements for each simulation

physical_all = []
comoving_all = []
snapshots_all = []
starting_values = []

simulations = [2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27, 28]
annuli = np.arange(0,0.007, 0.00034)

for i in range(len(simulations)):
    print(i)
    n = simulations[i]
    centres, snapshots, radii, stellar_mass, stellar_radius = load_simulation_data(n)

    starting_value = snapshot_10_percent(stellar_mass, snapshots)
    starting_values.append(starting_value)
    
    physical =[]
    comoving = []
    snaps = []
    for j in range(starting_value,128):
        snap = j
        snaps.append(snap)
        index = snapshots.index(snap)
        scale_factor = B.scale_fac(n, snap)
        
        coord2d, velocities, magnitudes, masses = load_aligned_data(n, snap)
        mag = magnitudes[:,3]    #k-band magnitude
        flux = 10**(-0.4*mag)
        coord2d = coord2d*scale_factor   #make coordinates physical
        distances = np.linalg.norm((coord2d), axis=1) 
    
        displacement = []
        for k in range(len(annuli)-1):
            idx, = np.where((distances >= annuli[k]) & (distances <= annuli[k+1]))
            all_coords = coord2d[idx,:]
            all_flux = flux[idx,]
            barycentre = A.ShrinkingSpheres.barycentre(all_coords, all_flux)
            distance = np.sqrt(barycentre[0]**2+barycentre[1]**2)
            displacement.append(distance)

        physical.append(max(displacement))
        comoving.append(max(displacement)/scale_factor)

    physical_all.append(physical)
    comoving_all.append(comoving)
    snapshots_all.append(snaps)

In [ ]:
for j in range(len(simulations)):
    A.util.save_file(np.array(snapshots_all[j]), 'Snaps_10kpc_{}'.format(simulations[j]))

In [ ]:
plt.plot(snapshots_all[0],physical_all[0])

In [ ]:
half_mass_rad = []

for i in range(len(simulations)):
    n = simulations[i]
    centres, snapshots, radii, stellar_mass, stellar_radius = load_simulation_data(n)

    half_mass_rad.append(2*stellar_radius[0])
half_mass_rad = np.array(half_mass_rad)/0.6777

In [ ]:
#LOAD IN DATA
#these are for 2*R_1/2, physical and comoving coordinates have NOT been /h yet!!!

starting_values =np.array([58, 56, 81, 64, 59, 76, 60, 58, 63, 70, 65, 62, 76, 62, 55, 56, 65,58, 75, 59, 59, 62, 57, 67, 63, 78])
simulations = [2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
physical = [np.load('/cosma5/data/durham/dc-keav1/Data/Physical_displacements/physical_{}.npy'.format(simulation)) for simulation in simulations]
comoving = [np.load('/cosma5/data/durham/dc-keav1/Data/Comoving_displacements/comoving_{}.npy'.format(simulation)) for simulation in simulations]
snapshots = [list(np.load('/cosma5/data/durham/dc-keav1/Data/Displacement_snapshots/snapshots_{}.npy'.format(simulation))) for simulation in simulations]

In [ ]:
#These are for R = 10 kpc, NOT been / h yet!

simulations = [2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
physical = [np.load('/cosma5/data/durham/dc-keav1/Data/Physical_10kpc/Physical_10kpc_{}.npy'.format(simulation)) for simulation in simulations]
comoving = [np.load('/cosma5/data/durham/dc-keav1/Data/Comoving_10kpc/Comoving_10kpc_{}.npy'.format(simulation)) for simulation in simulations]
snapshots = [list(np.load('/cosma5/data/durham/dc-keav1/Data/Snapshots_10kpc/Snaps_10kpc_{}.npy'.format(simulation))) for simulation in simulations]

In [ ]:
stellar_masses = [7.05, 7.75, 7.1, 6.72,4.75, 4.88, 2.99,6.1, 5.94, 6.01, 6.19, 10.39, 3.93, 5.41, 7.61, 8.04, 5.32, 4.74, 7.72, 6.02, 9.02, 6.55, 3.14, 10.97, 9.61, 10.45]
d_t = [0.73, 0.76, 0.65, 0.71, 0.67, 0.55, 0.79, 0.62, 0.43, .79, .31, .67, .73, .81, .36, .68, .71,.49,.83,.51,.63,.63,.73,.46,.81,.74]
r_d = [11.64, 7.26, 3.93, 3.58, 5.95,5.14,6.57,3.37,2.60,3.29,3.38, 4.19,5.32,9.03,4.19, 3.72,4.81,8.02,4.61, 2.25, 4.99, 5.57, 6.70,3.14,4.29,2.16]

In [ ]:
plt.figure(figsize= (5,5))
plt.hist(half_mass_rad*10**3, bins = 13, density = False)
plt.xlim(0, 30)
plt.xlabel(r'$2R_{1/2}\mathrm{\, / \,kpc}$', fontsize = 16)
plt.ylabel(r'$\mathrm{Number \, of \, simulations}$', fontsize = 16)

plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

In [ ]:
maximum_phys = []
maximum_comov = []

max_phys_snaps = []
max_comov_snaps = []

for j in range(len(snapshots)):
    distance = list(physical[j])
    maximum_phys.append(max(distance)/0.6777) 
    snap = snapshots[j][distance.index(max(distance))]
    max_phys_snaps.append(snap)
    distance = list(comoving[j])
    maximum_comov.append(max(distance)/0.6777) 
    snap = snapshots[j][distance.index(max(distance))]
    max_comov_snaps.append(snap)     #DIVIDE BY HUBBLE CONSTANT TO MAKE MEANINGFULL!

max_phys_times = [B.time(1,snap) for snap in max_phys_snaps]
max_comov_times = [B.time(1,snap) for snap in max_comov_snaps]

maximum_phys = np.array(maximum_phys)
maximum_comov = np.array(maximum_comov)

In [ ]:
for j in range(len(max_phys_times)):
    if max_phys_times[j] != max_comov_times[j]:
        print(simulations[j])

In [ ]:
plt.figure(figsize = (5,5), dpi = 300)
colors = ['black', 'black', 'black', 'red', 'black', 'black', 'black', 'red', 'red', 'black', 'black', 'black', 'red', 'black', 'red', 'red', 'black','black','black','red', 'black','red','black','black','red', 'black','black']
plt.scatter(max_comov_times, maximum_comov*10**3, marker = "o", s = 12, color = 'white')
# plt.yscale('log')

for i, (x, y) in enumerate(zip(max_comov_times,maximum_comov*10**3)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor = 'none',edgecolor=colors[i], boxstyle='circle,pad=0.2'))
plt.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 12)

xticks = [2,3,4,5,6,7,8,9,10]
plt.ylabel(r'$\mathrm{Maximum \, photocentre \, offset\, / \,ckpc}$', fontsize = 16)
plt.xlabel(r'$\mathrm{Time \, / \, Gyr}$', fontsize = 16)
plt.xticks(xticks, fontsize = 12)
plt.yticks(fontsize = 12)

plt.xlim(2,10)

In [ ]:
plt.figure(figsize = (5,5))
colors = ['black', 'black', 'black', 'red', 'black', 'black', 'black', 'red', 'red', 'black', 'black', 'black', 'red', 'black', 'red', 'red', 'black','black','black','red', 'black','red','black','black','red', 'black','black']
plt.scatter(max_phys_times, maximum_phys*10**3, marker = "o", s = 12, color = 'white')
# plt.yscale('log')

for i, (x, y) in enumerate(zip(max_phys_times,maximum_phys*10**3)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor = 'none',edgecolor=colors[i], boxstyle='circle,pad=0.2'))

plt.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 12)
plt.ylabel(r'$\mathrm{Maximum \, offset \, / \,kpc}$', fontsize = 16)
plt.xlabel(r'$\mathrm{Time \, / \, Gyr}$', fontsize = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.xlim(2,12)

In [ ]:
plt.figure(figsize = (5,5))

plt.scatter(stellar_masses, maximum_comov*10**3, marker = "o", s = 12, color = 'white')

for i, (x, y) in enumerate(zip(stellar_masses,maximum_comov*10**3)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor='none', edgecolor='black', boxstyle='circle,pad=0.3'))

plt.ylabel(r'$\mathrm{Displacement / ckpc}$', fontsize = 16)
plt.xlabel(r'$\mathrm{M_* \,/\, 10^{10} \, M_\odot}$', fontsize = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

In [ ]:
plt.figure(figsize = (5,5), dpi = 300)

plt.scatter(half_mass_rad*10**3, maximum_comov*10**3, marker = "o", s = 12, color = 'white')

for i, (x, y) in enumerate(zip(half_mass_rad*10**3,maximum_comov*10**3)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor='none', edgecolor='black', boxstyle='circle,pad=0.2'))

plt.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 12)
plt.ylabel(r'$\mathrm{Maximum\, photocentre \, offset\, / \,ckpc}$', fontsize = 16)
plt.xlabel(r'$2R_{1/2} \mathrm{ \, / \, kpc}$', fontsize = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.xlim(0,30)

In [ ]:
#MEDIAN COMOVING OFFSETS:

all_snaps = np.arange(0,128,1)
median_comov = []
median_phys = []
median_snaps = []
which_sims = []

for k in range(len(all_snaps)):
    snapshot = all_snaps[k]
    phys = []
    comov = []
    which_sim = []
    for j in range(len(simulations)):
        snapshots_sim = snapshots[j]
        if snapshot in snapshots_sim:
            phys.append(physical[j][snapshots_sim.index(snapshot)])
            comov.append(comoving[j][snapshots_sim.index(snapshot)])
            which_sim.append(simulations[j])

    if len(phys) >0:
        median_snaps.append(snapshot)
        median_phys.append(np.median(phys))
        median_comov.append(np.median(comov))
        which_sims.append(which_sim)

median_comov = np.array([med/0.6777 for med in median_comov])
median_phys = np.array([med/0.6777 for med in median_phys])
median_times = [B.time(1,snap) for snap in median_snaps]
len_sims = [len(which) for which in which_sims]

In [ ]:
redshifts = np.array([B.redshift(1, s) for s in median_snaps])

In [ ]:
ang_diam = A.util.angular_diameter_distance(redshifts)/0.6777*10**6   #returned in Mpc/h so convert into pc
angles = median_phys/ang_diam   #angle in radians
angle_arcsec = angles*206264.806*10**3

fig, ax1 = plt.subplots(figsize = (6,4))
# fig.dpi = 300
ax1.plot(median_times[:-1],angle_arcsec[:-1]*10**3)
ax1.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 16)
ax1.set_xlabel(r'$\mathrm{Time \, / \, Gyr}$', fontsize = 20)
ax1.set_ylabel(r'$\mathrm{\theta / \, \mu as}$', fontsize = 20)

ax1.set_xlim(3.1,14)
ax1.set_ylim(0,3.6)

In [ ]:
snapshots = [i for i in range(0,128)]
times = [B.time(1,snap) for snap in snapshots]

In [ ]:
ang_diams = []

for time in max_phys_times:
    index = times.index(time)
    snapshot = snapshots[index]
    redshift = B.redshift(1, snapshot)
    ang_diam = A.util.angular_diameter_distance(redshift)/0.6777*10**6   #returned in Mpc/h so convert into pc
    ang_diams.append(ang_diam)

ang_diams = np.array(ang_diams)
angles = maximum_phys/ang_diams
angle_arcsec = angles*206264.806*10**3
 
plt.figure(figsize = (5,5))
colors = ['black', 'black', 'black', 'red', 'black', 'black', 'black', 'red', 'red', 'black', 'black', 'black', 'red', 'black', 'red', 'red', 'black','black','black','red', 'black','red','black','black','red', 'black','black']
plt.scatter(max_phys_times, angle_arcsec*10**3, marker = "o", s = 12, color = 'white')
# plt.yscale('log')

for i, (x, y) in enumerate(zip(max_phys_times,angle_arcsec*10**3)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor = 'none',edgecolor=colors[i], boxstyle='circle,pad=0.2'))

plt.ylabel(r'$\mathrm{Angular\, offset \, / \,\mu as}$', fontsize = 16)
plt.xlabel(r'$\mathrm{Time \, / \, Gyr}$', fontsize = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.xlim(2,12)

In [ ]:
fig, ax1 = plt.subplots(figsize = (6,4))
# fig.dpi = 300
ax1.plot(median_times, median_comov*10**3)
ax1.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 16)
ax1.set_xlabel(r'$\mathrm{Time \, / \, Gyr}$', fontsize = 20)
ax1.set_ylabel(r'$\mathrm{Median \, offset \, / \, ckpc}$', fontsize = 20)

ax1.set_xlim(3.1,14)
ax1.set_ylim(0,9)

In [ ]:
fig, ax1 = plt.subplots(figsize = (6,4))
# fig.dpi = 300
ax1.plot(median_times, median_phys*10**3)
ax1.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 16)
ax1.set_xlabel(r'$\mathrm{Time \, / \, Gyr}$', fontsize = 20)
ax1.set_ylabel(r'$\mathrm{Median \, offset \, / \, kpc}$', fontsize = 20)

ax1.set_xlim(3.1,14)
ax1.set_ylim(0,2.7)

In [ ]:
#HISTOGRAMS
import scipy.stats as stats
snaps = [100]
plt.figure(figsize = (4,4))

for snap in snaps:

    phys = []
    comov = []
    
    for j in range(len(simulations)):
        snapshots_sim = snapshots[j]
        if snap in snapshots_sim:
            phys.append(physical[j][snapshots_sim.index(snapshot)])
            comov.append(comoving[j][snapshots_sim.index(snapshot)])
    
    phys = np.array([x/0.6777 for x in phys])
    comov = np.array([x/0.6777 for x in comov])
    
    plt.hist(np.array(comov)*10**3, bins=10, density = True, histtype = 'step', label = '{} Gyr'.format(np.round(B.time(1, snap),2)))
    data = np.array(comov)*10**3
    g =25
    l = 8

    kde = stats.gaussian_kde(data)
    x_vals = np.linspace(0, l, g)  # Define range for smooth curve
    # plt.plot(x_vals, kde(x_vals), label = '{} Gyr'.format(np.round(B.time(1, snap),2)))
        
plt.xlabel('Comoving displacement / ckpc')
plt.ylabel('Fraction')
plt.xlim(0,2)

plt.legend()


## Harmonic analysis

In [ ]:
def fourier_mode(m, coord, masses):
    angles = np.arctan2(coord[:,1],coord[:,0])
    result = masses*np.exp(1j * angles*m)
    
    A_m = np.abs(np.sum(result))/np.abs(np.sum(masses))
    return A_m

In [ ]:
m = 1
All_As = []
Times = []
simulations = [23,24,25,26,27,28]
annuli = np.arange(0, 0.007, 0.00034)

for j in range(len(simulations)):
    print(j)
    n = simulations[j]
    centres, snapshots, radii, stellar_mass, stellar_radius = load_simulation_data(n)
    starting_val = starting_values[simulations.index(n)]
    A_max = []
    times = []

    for snap in range(starting_val,128):
        coord2d, velocities, magnitudes, masses = load_aligned_data(n, snap)
        coord2d*=B.scale_fac(n,snap)
        distances = np.linalg.norm((coord2d), axis=1) 
        mag = magnitudes[:,3]    #k-band magnitude
        flux = 10**(-0.4*mag)
    
        As = []
        for k in range(len(annuli)-1):
            idx, = np.where((distances >= annuli[k]) & (distances <= annuli[k+1]))
            coord = coord2d[idx,:]
            magnitudes = mag[idx,]
            mass = masses[idx,]
            fluxes = flux[idx,]
            A_m = fourier_mode(m, coord,mass)
            As.append(A_m)
            
        A_max.append(max(As))
    
        times.append(B.time(n, snap))

    All_As.append(A_max)
    Times.append(times)

In [ ]:
#for when using 2R_1/2 for each simulation

starting_values =np.array([58, 56, 81, 64, 59, 76, 60, 58, 63, 70, 65, 62, 76, 62, 55, 56, 65,58, 75, 59, 59, 62, 57, 67, 63, 78])
simulations = [2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
timesA = [np.load('/cosma5/data/durham/dc-keav1/Data/A1_times/A1_time_{}.npy'.format(simulation)) for simulation in simulations]
AS = [np.load('/cosma5/data/durham/dc-keav1/Data/A1_max/A1_{}.npy'.format(simulation)) for simulation in simulations]

In [ ]:
#for when using R = 10 kpc for each simulation

starting_values = np.load('Data/Starting_values.npy')
simulations = [2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
timesA = [np.load('Data/As_times_10kpc/As_times_10kpc_{}.npy'.format(simulation)) for simulation in simulations]
AS = [np.load('Data/As_10kpc/As_10kpc_{}.npy'.format(simulation)) for simulation in simulations]

In [ ]:
max_A1 = []
max_time = []

for j in range(len(timesA)):
    A1 = list(AS[j])
    max_A1.append(max(A1)) 
    time = timesA[j][A1.index(max(A1))]
    max_time.append(time)

In [ ]:
plt.figure(figsize = (5,5), dpi = 300)
plt.scatter(max_time,max_A1, marker = "o", s = 12, color = 'white')
xticks = [2,3,4,5,6,7,8,9,10,11,12]
colors = ['black', 'black', 'black', 'red', 'black', 'black', 'black', 'red', 'red', 'black', 'black', 'black', 'red', 'black', 'red', 'red', 'black','black','black','red', 'black','red','black','black','red', 'black','black']
for i, (x, y) in enumerate(zip(max_time,max_A1)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor='none', edgecolor=colors[i], boxstyle='circle,pad=0.2'))
plt.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 12)

plt.ylabel(r'$\mathrm{Maximum \, A_1}$', fontsize = 16)
plt.xlabel(r'$\mathrm{Time \, / \, Gyr}$', fontsize = 16)
plt.xticks(xticks, fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlim(2,12)

In [ ]:
plt.figure(figsize = (5,5), dpi = 300)

plt.scatter(half_mass_rad*10**3,max_A1, marker = "o", s = 12, color = 'white')

for i, (x, y) in enumerate(zip(half_mass_rad*10**3,max_A1)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor='none', edgecolor='black', boxstyle='circle,pad=0.2'))
plt.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 12)
plt.ylabel(r'$\mathrm{Maximum \, A_1}$', fontsize = 16)
plt.xlabel(r'$2R_{1/2}\mathrm{\, / \,kpc}$', fontsize = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlim(0,30)

In [ ]:
plt.figure(figsize = (5,5))

plt.scatter(maximum_comov*10**3,max_A1, marker = "o", s = 12, color = 'white')

for i, (x, y) in enumerate(zip(maximum_comov*10**3,max_A1)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor='none', edgecolor=colors[i], boxstyle='circle,pad=0.2'))

plt.ylabel(r'$\mathrm{A_1}$', fontsize = 16)
plt.xlabel(r'$\mathrm{Maximum\, comoving \, offset \,/ \,ckpc}$', fontsize = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

In [ ]:
all_snaps = np.arange(0,128)
medianA = []
median_times = []
which_sims = []

for k in range(len(all_snaps)):
    snapshot = all_snaps[k]
    time = B.time(1,snapshot)
    A1 = []
    which_sim = []
    for j in range(len(simulations)):
        if time in timesA[j]:
            A1.append(AS[j][list(timesA[j]).index(time)])
            which_sim.append(simulations[j])

    if len(A1) >0:
        median_times.append(time)
        medianA.append(np.median(A1))
        which_sims.append(which_sim)

len_sims = [len(which) for which in which_sims]

In [ ]:
fig, ax1 = plt.subplots(figsize = (6,4))
ax1.plot(median_times, medianA)
ax1.tick_params(axis = 'both', top = True, right = True,direction = 'in',labelsize = 16)
ax1.set_xlabel(r'$\mathrm{Time \, / \, Gyr}$', fontsize = 20)
ax1.set_ylabel(r'$\mathrm{Median\, A_1}$', fontsize = 20)

ax1.set_xlim(3,14)
ax1.set_ylim(0,0.25)

In [ ]:
plt.figure(figsize = (5,5),dpi = 300)

plt.scatter(max_time, max_phys_times, marker = "o", s = 12, color = 'white')

for i, (x, y) in enumerate(zip(max_time, max_phys_times)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor='none', edgecolor=colors[i], boxstyle='circle,pad=0.2'))

plt.ylabel(r'$\mathrm{Physical \, maximum \, time \, / \, Gyr}$', fontsize = 16)
plt.xlabel(r'$\mathrm{Asymmetry \, maximum \, time \, / \, Gyr}$', fontsize = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlim(2.5, 12)
plt.ylim(2.5, 12)

In [ ]:
plt.figure(figsize = (5,5))

plt.scatter(max_phys_times, max_comov_times, marker = "o", s = 12, color = 'white')

for i, (x, y) in enumerate(zip(max_phys_times, max_comov_times)):
    plt.text(x, y, str(simulations[i]), fontsize=10, ha='center', va='center', color='black',
             bbox=dict(facecolor='none', edgecolor=colors[i], boxstyle='circle,pad=0.2'))

plt.ylabel(r'$\mathrm{Comoving \, maximum \, time \, / \, Gyr}$', fontsize = 16)
plt.xlabel(r'$\mathrm{Physical \, maximum \, time \, / \, Gyr}$', fontsize = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlim(2.5, 12)
plt.ylim(2.5, 12)

In [ ]:
for j in range(len(max_phys_times)):
    if max_phys_times[j] != max_time[j]:
        print(simulations[j])

### Stellar density Histogram

In [ ]:
filenames = []
count = 0

b = 0.035
simulation_number = 24


for i in range(105, 135):

    snap = i
# snap = 70

    time = A.Others.time(simulation_number, snap)
    n = simulations.index(simulation_number)
    
    Centre = centres[n]
    snaps = snapshots[n]
    rad = radii[n]
    
    index = list(snaps).index(snap)
    
    results =  A.Others.load_snapshot(simulation_number, snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses"])
    coordinates, magnitudes, velocities, masses = results.get_results()
    radius = rad[index]
    centre = Centre[index]

    # new_coords, new_vels, coords, vels, masses, magnitude = A.util.align(centre, coordinates, velocities, masses, radius, 0.5, idx = None, other = [magnitudes], to_print = False)
    new_coords = A.util.CentreOnHalo(coordinates, centre)
    
    n, xedges, yedges = np.histogram2d(new_coords[:,1], new_coords[:,2], bins=(500,500), range = [[-b,b], [-b, b]])
    xbin = 0.5 * (xedges[:-1] + xedges[1:])
    ybin = 0.5 * (yedges[:-1] + yedges[1:])
    xc, yc = np.meshgrid(xbin, ybin)
    fig, ax1 = plt.subplots(figsize=(4,4))
    ax1.set_aspect(1)
    ax1.contourf( xc*1000, yc*1000, np.log10(n.T), cmap='magma')
    
    plt.suptitle('Auriga {}'.format(simulation_number), fontsize = 14)
    plt.title('Time = {} Gyr'.format(np.round(time,2)), fontsize = 12)
    plt.xlabel('kpc', fontsize = 12)
    plt.ylabel('kpc', fontsize = 12)

    count += 1
    
    name = 'Images_for_gif/{}.png'.format(count)
    plt.savefig(name)
    filenames.append(name)

    plt.close()

In [ ]:
with imageio.get_writer('Auriga{}.gif'.format(simulation_number), mode='I', fps = 2) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

### Find rotation matrices for each one

In [ ]:
rotationmatrices = []
simulation_number = 24
times = []
rotation_snaps = []

for i in range(35, 252):
    snap = i
    if snap%50 ==0:
        print(snap)
    
    time = A.Others.time(simulation_number, snap)
    n = simulations.index(simulation_number)
    
    Centre = centres[n]
    snaps = snapshots[n]
    rad = radii[n]

    if snap in snaps:
        index = list(snaps).index(snap)
        
        results =  A.Others.load_snapshot(simulation_number, snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses"])
        coordinates, magnitudes, velocities, masses = results.get_results()
        radius = rad[index]
        centre = Centre[index]
    
    
        coords = A.util.CentreOnHalo(coordinates, centre)
        vels, bulk_velocity = A.util.remove_bulk_velocity(coords, velocities, masses, radialcut=0.2*radius)
        
        rad = A.util.r(coords)
        idx, = np.where(rad < 0.1*radius)
        
        if len(idx,)>0:
            # Calculate angular momentum (L)
            L = np.cross(coords[idx, :], vels[idx, :] * masses[idx, None])
            Ltot = L.sum(axis=0)
            Ldir = Ltot / np.sqrt((Ltot**2).sum())
    
            # Get principal axes and rotation matrix
            xdir, ydir, zdir = A.util.get_principal_axis(coords, masses, idx, L=Ldir)
            
            matrix = np.array([xdir, ydir, zdir])
            
            rotmat = matrix.transpose()  # Transpose to align correctly
            rotationmatrices.append(rotmat)
            times.append(time)
            rotation_snaps.append(snap)

In [ ]:
from scipy.spatial.transform import Rotation as R

euler_angles = np.array([R.from_matrix(rotmat).as_euler('xyz', degrees=True)  for rotmat in rotationmatrices])

plt.figure(figsize = (4,4))
plt.plot(times, euler_angles[:,0], label = 'x angle')
plt.plot(times, euler_angles[:,1], label = 'y angle')
plt.plot(times, euler_angles[:,2], label = 'z angle')

plt.xlabel('Time / Gyr')
plt.ylabel('Angle / degrees')

plt.legend()
plt.show()

In [ ]:
C = rotationmatrices[1]
B = rotationmatrices[-1]

n = len(times)

matrices = [(1 - t) * C + t * B for t in np.linspace(0, 1, n)]

In [ ]:
euler_angles = np.array([R.from_matrix(rotmat).as_euler('xyz', degrees=True)  for rotmat in matrices])

plt.figure(figsize = (4,4))

plt.plot(times, euler_angles[:,0], label = 'x angle')
plt.plot(times, euler_angles[:,1], label = 'y angle')
plt.plot(times, euler_angles[:,2], label = 'z angle')

plt.xlabel('Time / Gyr')
plt.ylabel('Angle / degrees')

plt.legend()
plt.show()

In [ ]:
len(rotationmatrices)

In [ ]:
def stellar_hist_rot(simulation_number, snap, histogram_width, axes):

    time = A.Others.time(simulation_number, snap)
    
    b = histogram_width
    n = simulations.index(simulation_number)
    
    Centre = centres[n]
    snaps = snapshots[n]
    rad = radii[n]
    
    index = list(snaps).index(snap)

    results =  A.Others.load_snapshot(simulation_number, snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses"])
    coordinates, magnitudes, velocities, masses = results.get_results()
    radius = rad[index]
    centre = Centre[index]


    if snap in rotation_snaps:
        rotationmatrices[rotation_snaps.index(snap)]
        coords = A.util.CentreOnHalo(coordinates, centre)
        vels, bulk_velocity = A.util.remove_bulk_velocity(coords, velocities, masses, radialcut=0.1*radius)
        
        rad = A.util.r(coords)
        idx, = np.where(rad < 0.1*radius)

        new_coords = np.dot(coords, rotmat)
        new_vels = np.dot(vels, rotmat)

    else:
        new_coords, new_vels, coords, vels, masses, magnitude = A.util.align(centre, coordinates, velocities, masses, radius, 0.5, idx = None, other = [magnitudes], to_print = False)

    n, xedges, yedges = np.histogram2d(new_coords[:,axes[0]], new_coords[:,axes[1]], bins=(500,500), range = [[-b,b], [-b, b]])
    xbin = 0.5 * (xedges[:-1] + xedges[1:])
    ybin = 0.5 * (yedges[:-1] + yedges[1:])
    xc, yc = np.meshgrid(xbin, ybin)
    fig, ax1 = plt.subplots(figsize=(4,4))
    ax1.set_aspect(1)
    ax1.contourf( xc*1000, yc*1000, np.log10(n.T), cmap='magma')

    plt.suptitle('Auriga {}'.format(simulation_number), fontsize = 14)
    plt.title('Time = {} Gyr'.format(np.round(time,2)), fontsize = 12)
    plt.xlabel('kpc', fontsize = 12)
    plt.ylabel('kpc', fontsize = 12)


In [ ]:
b = 0.035
simulation_number = 9

filenames = []
count = 0

for i in range(15,251,6):
    snap = i
    n = simulations.index(simulation_number)
    
    Centre = centres[n]
    snaps = snapshots[n]
    rad = radii[n]
    
    index = list(snaps).index(snap)

    results =  A.Others.load_snapshot(simulation_number, snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses"])
    coordinates, magnitudes, velocities, masses = results.get_results()
    radius = rad[index]
    centre = Centre[index]

    if snap in rotation_snaps:
        rotationmatrices[rotation_snaps.index(snap)]
        coords = A.util.CentreOnHalo(coordinates, centre)
        vels, bulk_velocity = A.util.remove_bulk_velocity(coords, velocities, masses, radialcut=0.1*radius)
        
        rad = A.util.r(coords)
        idx, = np.where(rad < 0.1*radius)

        new_coords = np.dot(coords, rotmat)
        new_vels = np.dot(vels, rotmat)

    else:
        new_coords, new_vels, coords, vels, masses, magnitude = A.util.align(centre, coordinates, velocities, masses, radius, 0.5, idx = None, other = [magnitudes], to_print = False)

    n, xedges, yedges = np.histogram2d(new_coords[:,1], new_coords[:,2], bins=(500,500), range = [[-b,b], [-b, b]])
    xbin = 0.5 * (xedges[:-1] + xedges[1:])
    ybin = 0.5 * (yedges[:-1] + yedges[1:])
    xc, yc = np.meshgrid(xbin, ybin)
    fig, ax1 = plt.subplots(figsize=(4,4))
    ax1.set_aspect(1)
    ax1.contourf( xc*1000, yc*1000, np.log10(n.T), cmap='magma')

    plt.suptitle('Auriga {}'.format(simulation_number), fontsize = 14)
    plt.xlabel('kpc', fontsize = 12)
    plt.ylabel('kpc', fontsize = 12)

    count += 1
    
    name = 'Images_for_gif/{}.png'.format(count)
    plt.savefig(name)
    filenames.append(name)

    plt.close()

In [ ]:
with imageio.get_writer('Auriga{}_rotmat.gif'.format(simulation_number), mode='I', fps = 2) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

In [ ]:
stellar_hist(24, 150, 0.035,[1,2])

In [ ]:
stellar_hist_rot(24, 150, 0.035,[1,2])